In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
from functions import base_functions
from tqdm import tqdm
import math
from sklearn.linear_model import LinearRegression
from SymbolicNet import SymbolicNet
import pandas as pd
import numpy as np

In [2]:
x = torch.tensor(pd.read_csv("x_and_u.csv", header=None).to_numpy(dtype=np.float32))
y = torch.tensor(pd.read_csv("x_dot.csv", header=None).to_numpy(dtype=np.float32))

y = y[:, 2].view(-1, 1)

In [3]:
features = base_functions

# Engineer features
x_engineered = torch.stack([feature.torch(x) for feature in features]).view(x.shape[0], -1)

In [4]:
x_engineered.shape

torch.Size([49990, 52])

In [5]:
x_standard = (x_engineered - x_engineered.mean(0)) / x_engineered.std(0)
y_standard = (y - y.mean(0)) / y.std(0)

In [6]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()

In [7]:
linear_model.fit(x_standard, y_standard)

LinearRegression()

In [8]:
thetas = linear_model.coef_[0]

In [9]:
thetas

array([-2.5949872e-01, -6.3618416e-01,  5.9650415e-01, -2.0713511e+00,
        2.5679645e-01,  1.4329413e+00,  3.1420181e+00, -1.4134698e+00,
        4.2699356e-02, -2.2646286e-02,  3.3303171e-02, -3.8665012e-03,
       -7.8400016e-02,  6.5291382e-02,  7.0747560e-01, -1.1085117e-01,
        1.9861752e+00, -1.2607634e-01,  1.7179877e-02, -2.8282943e+00,
        1.4058053e-01, -1.4824159e-03, -4.1719824e-03,  3.1474665e-02,
       -2.0910949e-03,  1.6422617e-01,  2.9854088e+00, -8.2631707e-03,
        1.5450239e+00, -1.3057619e+00, -4.5983839e-01, -6.3892460e-01,
       -9.3637508e-01,  1.0625193e+00, -4.2167157e-03,  7.7973545e-02,
        6.4849377e-02,  4.7801882e-03, -9.8425895e-02, -2.7902851e+00,
       -7.2162628e-02, -2.0475473e+00,  1.4124613e+00,  3.6067393e-01,
       -8.3072633e-01,  6.1784697e-01,  2.1680194e-01,  1.8937886e-04,
       -5.1876605e-02,  3.7079848e-02, -7.6976791e-04,  4.9345791e-03],
      dtype=float32)

In [10]:
# Create list where features are sorted by importance
features = [[feature, importance] for feature, importance in zip(features, thetas)]
features.sort(key=lambda x: abs(x[1]), reverse=True)

In [11]:
features

[[<functions.Cos at 0x7f21662da080>, -2.071351],
 [<functions.Linear at 0x7f21662d9fc0>, -0.63618416],
 [<functions.Sin at 0x7f21662da020>, 0.59650415],
 [<functions.Square at 0x7f21662d9f60>, -0.25949872]]

In [12]:
from functions import Constant

#optimal_features = [feature for feature in features if feature[1] > 0.05 or type(feature[0]) == type(Constant())]
optimal_features = [feature for feature in features[:2]]

In [13]:
functions = [feature[0] for feature in optimal_features]

print(f"{[func.sympy() for func in functions]} are to most important features.")

[cos(x), 'x'] are to most important features.


In [14]:
model = SymbolicNet(13, 13, 1, functions)

NameError: name 'lem' is not defined

In [ ]:
model.train(True)
model.train_n_epochs(x, y, epochs=100, lr=0.01)

  1%|          | 1/100 [00:15<25:39, 15.55s/it]

Epoch: 0 Loss: 0.006249740719795227


  5%|▌         | 5/100 [01:32<29:20, 18.53s/it]


KeyboardInterrupt: 

In [ ]:
weights = model.get_weights()

In [ ]:
print(weights)

Parameter containing:
tensor([[-3.6956, -0.1252],
        [-0.1304, -0.1776],
        [-3.5664, -0.4498],
        [-6.4716,  0.3775],
        [-0.1225, -0.0254],
        [ 0.4328, -0.0877],
        [ 2.5887,  0.0486],
        [-1.9826,  0.1749],
        [-0.3090,  0.1626],
        [ 7.7587,  0.2802],
        [ 3.5037,  1.1880],
        [-4.2191, -0.0695],
        [-0.0221, -0.1349]], requires_grad=True)


In [ ]:
for w, f in zip(weights[0], model.functions):
    print(w.item(), f.sympy())
    

-0.20419195294380188 x**2
-0.08320192992687225 x
